# Setting environment parameter

In [1]:
import os
os.environ["OPENAI_API_KEY"]=""


In [2]:
#!pip install -U langchain-chroma

In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader

CHROMA_PATH = r".\chroma5"


# load data into Chroma DB

In [4]:
# embeddings = OpenAIEmbeddings()
loaders = PyPDFLoader(r"D:\Generative AI\Bhagavad-gita-Swami-BG-Narasingha.pdf")

documents = loaders.load()

db = Chroma.from_documents(
    documents,
    OpenAIEmbeddings(),
    persist_directory=CHROMA_PATH
  )
db.persist()


C:\Users\Sreedhar\AppData\Local\Temp\ipykernel_3992\3073452048.py:11: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [5]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
 - -
Answer the question based on the above context: {question}
"""

In [6]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [7]:
def query_rag(query_text):
  """
  Query a Retrieval-Augmented Generation (RAG) system using Chroma database and OpenAI.
  Args:
    - query_text (str): The text to query the RAG system with.
  Returns:
    - formatted_response (str): Formatted response including the generated text and sources.
    - response_text (str): The generated response text.
  """
  # YOU MUST - Use same embedding function as before
  embedding_function = OpenAIEmbeddings()

  # Prepare the database
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)
  
  # Retrieving the context from the DB using similarity search
  results = db.similarity_search_with_relevance_scores(query_text, k=3)

  # Check if there are any matching results or if the relevance score is too low
  # if len(results) == 0 or results[0][1] < 0.7:
  #   print(f"Unable to find matching results.")

  # Combine context from matching documents
  context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in results])
 
  # Create prompt template using context and query text
  prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
  prompt = prompt_template.format(context=context_text, question=query_text)
  
  # Initialize OpenAI chat model
  model = ChatOpenAI()

  # Generate response text based on the prompt
  response_text = model.predict(prompt)
 
   # Get sources of the matching documents
  sources = [doc.metadata.get("source", None) for doc, _score in results]
 
  # Format and return response including generated text and sources
  formatted_response = f"Response: {response_text}\nSources: {sources}"
  return formatted_response, response_text


# Ask Question

In [8]:
query_text="what krishna told to Arjuna in chapter 2"
formatted_response, response_text = query_rag(query_text)
print(response_text)

C:\Users\Sreedhar\AppData\Local\Temp\ipykernel_3992\3764950876.py:14: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)
C:\Users\Sreedhar\AppData\Local\Temp\ipykernel_3992\3764950876.py:34: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response_text = model.predict(prompt)


Based on the context provided, Krishna explained the greatest secret about His supreme nature to Arjuna in Chapter 2.


In [9]:
query_text="What is kurukshetra?"
formatted_response, response_text = query_rag(query_text)
print(response_text)

Kurukshetra was a fratricidal war – a family feud brought on by human faults such as greed for sovereignty, failure in duty, envy, family attachment, and falsely identifying the body as the self.


In [10]:
query_text="Who is Arjuna?"
formatted_response, response_text = query_rag(query_text)
print(response_text)

Arjuna is a character from the Bhagavad-gita, to whom the Supreme Controller, Lord Krishna, is giving encouragement and guidance.


In [11]:
query_text="FOr which Arjuna is popular for?"
formatted_response, response_text = query_rag(query_text)
print(response_text)

Arjuna is popular for his natural propensity as a warrior and his desire to see Duryodhana disgraced.


In [12]:
query_text="FOr which skill Arjuna is popular for?"
formatted_response, response_text = query_rag(query_text)
print(response_text)

Arjuna is popular for his skill in facing the enemy as a duty of a Kshatriya.


In [13]:
query_text="Provide me names of Pandavas?"
formatted_response, response_text = query_rag(query_text)
print(response_text)

The names of the Pandavas are Yudhishthira, Bhima, Arjuna, Nakula, and Sahadeva.


# 1. get the embeddings for the query
# 2. get the context from the chroma related to embeddings
# 3. formulate the context based on query
# 4. Ask the llm with the  context and query

In [14]:
query_text="what are various yogas as mentioned in Bhagavadgita"
formatted_response, response_text = query_rag(query_text)
print(response_text)

The various yogas mentioned in Bhagavad-gita include bhakti-yoga, karma-yoga, jnana-yoga, and dhyana-yoga.


In [15]:
query_text="what did krishna told to karna"
formatted_response, response_text = query_rag(query_text)
print(response_text)

Based on the context provided, there is no mention of Krishna speaking to Karna. Therefore, it cannot be determined what Krishna told to Karna.


In [16]:
query_text="For which skill karna popular for"
formatted_response, response_text = query_rag(query_text)
print(response_text)

In the context provided, Karna is popular for his skill in archery.


In [17]:
query_text="Why did karna chose kauravas"
formatted_response, response_text = query_rag(query_text)
print(response_text)

Karna chose to support the Kauravas because he was a close friend of Duryodhana and felt indebted to him for giving him a kingdom and recognition despite his low birth.


In [20]:
query_text="Who is the guru of karna"
formatted_response, response_text = query_rag(query_text)
print(response_text)

Based on the context provided, it does not explicitly mention who the guru of Karna is.


In [21]:
query_text="Who is the guru of pandavas and kauravas"
formatted_response, response_text = query_rag(query_text)
print(response_text)

The guru of the Pandavas and Kauravas is mentioned as the great sage Vyasa in the context provided.


In [22]:
query_text="What is the guru name of pandavas and kauravas"
formatted_response, response_text = query_rag(query_text)
print(response_text)

The guru name of the Pandavas and Kauravas is Çrî K®ß√a.


In [23]:
query_text="How long did the war happened in Kurukshetra"
formatted_response, response_text = query_rag(query_text)
print(response_text)

The war in Kurukshetra lasted for 18 days.


In [24]:
query_text="How many people died in the kurukshetra war"
formatted_response, response_text = query_rag(query_text)
print(response_text)

The exact number of people who died in the Kurukshetra war is not specified in the provided context.


In [25]:
query_text="How did karna died"
formatted_response, response_text = query_rag(query_text)
print(response_text)

Karna died in the battle of Kurukshetra after being fatally wounded by Arjuna.


In [26]:
query_text="What happend to krishna after the kurukshetra war"
formatted_response, response_text = query_rag(query_text)
print(response_text)

After the Kurukshetra war, Krishna eventually passed away.


In [27]:
query_text="Who is the father of pandavas"
formatted_response, response_text = query_rag(query_text)
print(response_text)

The father of the På√∂avas is King Kuru.


In [28]:
query_text="Who is the father of pandavas"
formatted_response, response_text = query_rag(query_text)
print(response_text)

The father of the Pandavas is King Kuru.


In [29]:
query_text="Why did balarama did not involve in the war"
formatted_response, response_text = query_rag(query_text)
print(response_text)

Balarama did not involve in the war because he did not want to take sides in the conflict between good and evil, as mentioned in the context that wars were fought to resolve the differences between good and evil.


In [30]:
query_text="Who is the brother of balarama"
formatted_response, response_text = query_rag(query_text)
print(response_text)

The brother of Balarama is Lord Krishna.


In [31]:
query_text="Who raised abhimanyudu"
formatted_response, response_text = query_rag(query_text)
print(response_text)

Abhimanyudu was raised by his mother, Subhadra, who was the sister of Lord Krishna.


In [32]:
query_text="Where was abhimanyudu raised"
formatted_response, response_text = query_rag(query_text)
print(response_text)

Abhimanyu was raised in the spiritual realm of Ayodhyå, the realm of Råma, the avatåra of K®ß√a.


In [33]:
query_text="Who is the mother of karna"
formatted_response, response_text = query_rag(query_text)
print(response_text)

Based on the context provided, Karna's mother is not mentioned. The verses from the Bhagavad-gita focus on the vast expanse of material nature being the womb from which all living beings manifest, and the seed-giving father being the ultimate source of life.


In [34]:
query_text="How many are kauravas"
formatted_response, response_text = query_rag(query_text)
print(response_text)

There were a total of 100 Kauravas mentioned in the context.
